# Advanced Short Straddle Strategy - Real Market Data

This notebook implements an institutional-grade short straddle strategy with:
- Real historical data from Dolt database
- IV rank and trend analysis
- Advanced risk management
- Comprehensive visualizations
- 30+ performance metrics

## Environment Setup

**RECOMMENDED:** Launch Jupyter from the activated virtual environment for best performance.

**Option 1: From Virtual Environment (Recommended)**
```bash
cd /Users/janussuk/Desktop/OptionsBacktester2/code
source .venv/bin/activate
jupyter notebook ../notebooks/ShortStraddleStrategy_RealData.ipynb
```

**Option 2: From Any Jupyter Instance**
If you're using Anaconda or another Jupyter installation:
- The cell below will **automatically install** missing dependencies (doltpy, etc.)
- No manual setup required - just run the verification cell

The verification cell below will check your environment and install dependencies as needed.

In [112]:
# Environment Verification and Setup Cell
import sys
import os
from pathlib import Path
import subprocess

print("=" * 80)
print("ENVIRONMENT VERIFICATION & AUTO-SETUP")
print("=" * 80)

# 1. Python executable and version
print(f"\n1. Python Executable:")
print(f"   {sys.executable}")
print(f"\n2. Python Version:")
print(f"   {sys.version}")

# 3. Virtual environment check
venv_path = os.environ.get('VIRTUAL_ENV', None)
if venv_path:
    print(f"\n3. Virtual Environment:")
    print(f"   ✅ ACTIVE: {venv_path}")
else:
    print(f"\n3. Virtual Environment:")
    print(f"   ⚠️  NOT DETECTED")
    print(f"   Note: You can still use this notebook, dependencies will be installed automatically.")

# 4. Verify backtester package is available
print(f"\n4. Backtester Package:")
try:
    import backtester
    print(f"   ✅ FOUND (installed): {backtester.__file__}")
except ImportError:
    # If not installed, try adding to path
    code_path = Path('..') / 'code'
    if code_path.exists():
        sys.path.insert(0, str(code_path.absolute()))
        try:
            import backtester
            print(f"   ✅ FOUND (via path): {code_path.absolute()}")
        except ImportError:
            print(f"   ❌ NOT FOUND - Check ../code directory")
    else:
        print(f"   ❌ NOT FOUND - Installation required")

# 5. Check and auto-install dependencies
print(f"\n5. Dependencies (auto-installing if missing):")
dependencies = {
    'pandas': 'pandas',
    'numpy': 'numpy',
    'scipy': 'scipy',
    'plotly': 'plotly',
    'doltpy': 'doltpy'
}

all_ok = True
for name, module in dependencies.items():
    try:
        __import__(module)
        print(f"   ✅ {name}")
    except ImportError:
        print(f"   ⚠️  {name} - INSTALLING...")
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", module, "-q"])
            __import__(module)
            print(f"   ✅ {name} - INSTALLED SUCCESSFULLY")
        except Exception as e:
            print(f"   ❌ {name} - INSTALLATION FAILED: {e}")
            all_ok = False

# 6. Database path check
print(f"\n6. Database Path:")
db_path = Path('/Users/janussuk/Desktop/OptionsBacktester2/dolt_data/options')
if db_path.exists():
    print(f"   ✅ Found: {db_path}")
else:
    print(f"   ❌ Not found: {db_path}")
    all_ok = False

# Summary
print(f"\n" + "=" * 80)
if all_ok:
    print("✅ ENVIRONMENT READY - All checks passed!")
    print("You may proceed with running the notebook cells.")
else:
    print("⚠️  PARTIAL SETUP - Some dependencies may be missing")
    print("Try installing manually: pip install pandas numpy scipy plotly doltpy")
print("=" * 80)

ENVIRONMENT VERIFICATION & AUTO-SETUP

1. Python Executable:
   /Library/Developer/CommandLineTools/usr/bin/python3

2. Python Version:
   3.9.6 (default, Dec  2 2025, 07:27:58) 
[Clang 17.0.0 (clang-1700.6.3.2)]

3. Virtual Environment:
   ⚠️  NOT DETECTED
   Note: You can still use this notebook, dependencies will be installed automatically.

4. Backtester Package:
   ✅ FOUND (installed): /Users/janussuk/Desktop/AI Projects/OptionsBacktester2/notebooks/../code/backtester/__init__.py

5. Dependencies (auto-installing if missing):
   ✅ pandas
   ✅ numpy
   ✅ scipy
   ✅ plotly
   ✅ doltpy

6. Database Path:
   ❌ Not found: /Users/janussuk/Desktop/OptionsBacktester2/dolt_data/options

⚠️  PARTIAL SETUP - Some dependencies may be missing
Try installing manually: pip install pandas numpy scipy plotly doltpy


## 1. Setup and Imports

In [113]:
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Add backtester to path
sys.path.insert(0, str(Path('..') / 'code'))

from datetime import datetime, timedelta
import pandas as pd
import numpy as np

# Backtester framework imports
from backtester.data.dolt_adapter import DoltAdapter
from backtester.engine.backtest_engine import BacktestEngine
from backtester.engine.execution import ExecutionModel
from backtester.engine.data_stream import DataStream
from backtester.strategies.strategy import Strategy
from backtester.structures.straddle import ShortStraddle
from backtester.core.option_structure import OptionStructure

# Analytics imports
from backtester.analytics.metrics import PerformanceMetrics
from backtester.analytics.risk import RiskAnalytics
from backtester.analytics.visualization import Visualization

# Visualization
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

print('✓ All imports successful!')
print(f'Notebook created: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}')

✓ All imports successful!
Notebook created: 2025-12-21 20:16:16


## 2. Data Connection

In [114]:
# Database configuration
DB_PATH = '/Users/janussuk/Desktop/AI Projects/OptionsBacktester2/dolt_data/options'
UNDERLYING = 'SPY'
START_DATE = datetime(2023, 1, 4)
END_DATE = datetime(2025, 10, 29)
INITIAL_CAPITAL = 100000.0

# Connect to database
print(f'Connecting to Dolt database at: {DB_PATH}')
adapter = DoltAdapter(DB_PATH)
adapter.connect()

# Verify database
tables = adapter.get_tables()
print(f'✓ Database connected!')
print(f'Available tables: {tables}')

# Check data availability
date_range = adapter.get_date_range(UNDERLYING)
print(f'Data range for {UNDERLYING}: {date_range[0]} to {date_range[1]}')

Connecting to Dolt database at: /Users/janussuk/Desktop/AI Projects/OptionsBacktester2/dolt_data/options
✓ Database connected!
Available tables: ['option_chain', 'volatility_history']
Data range for SPY: 2019-02-09 00:00:00 to 2025-12-12 00:00:00


## 3. Strategy Implementation

Advanced Short Straddle with:
- IV rank-based entry (high IV)
- Dynamic profit targets (50% max profit)
- Stop loss management (2x max profit)
- DTE-based exits

In [115]:
class AdvancedShortStraddleStrategy(Strategy):
    """
    Advanced Short Straddle Strategy with IV-based entry and dynamic exits.
    
    Entry Rules:
    - IV rank > 50 (high implied volatility)
    - DTE between 3-70 days
    - No more than 10 concurrent positions
    
    Exit Rules:
    - 75% profit target
    - 200% stop loss (2x max profit)
    - Exit at 3 DTE or earlier
    """
    
    def __init__(self, name='AdvStraddle', initial_capital=100000.0):
        super().__init__(
            name=name,
            initial_capital=initial_capital,
            position_limits={
                'max_positions': 10,
                'max_total_delta': 5000.0,
                'max_total_vega': 3000.0,
                'max_capital_utilization': 0.9,
                'max_single_position_size': initial_capital * 0.5
            }
        )
        
        # Strategy parameters
        self.iv_rank_threshold = 40.0  # Enter when IV rank > 40 
        self.min_entry_dte = 3
        self.max_entry_dte = 70
        self.exit_dte = 3  # Exit at or before 3 DTE
        self.profit_target_pct = 0.75  # 75% of max profit
        self.stop_loss_pct = 2.00  # 200% of max profit (loss)
        self.contracts_per_trade = 1
        
        # Track IV history for rank calculation
        self.iv_history = []
        self.iv_lookback = 90  # days for IV rank calculation
        
    def calculate_iv_rank(self, current_iv):
        """Calculate IV rank (percentile over lookback period)."""
        if len(self.iv_history) < 10:
            return 50.0  # Default to mid-range if insufficient data
        
        iv_values = list(self.iv_history) + [current_iv]
        percentile = (np.searchsorted(sorted(iv_values), current_iv) / len(iv_values)) * 100
        return percentile
    
    def should_enter(self, market_data):
        """Determine if we should enter a new position."""
        # Update IV history
        current_iv = market_data.get('iv', 0.20)
        self.iv_history.append(current_iv)
        if len(self.iv_history) > self.iv_lookback:
            self.iv_history.pop(0)
        
        # Calculate IV rank
        iv_rank = self.calculate_iv_rank(current_iv)
        
        # Entry conditions
        high_iv = iv_rank > self.iv_rank_threshold
        max_positions_ok = self.num_open_positions < self._position_limits['max_positions']
        
        return high_iv and max_positions_ok
    
    def should_exit(self, structure, market_data):
        """Determine if we should exit a position."""
        # Calculate days to expiration
        current_date = market_data.get('date', datetime.now())
        if hasattr(structure, 'call_option'):
            expiration = structure.call_option.expiration
            dte = (expiration - current_date).days
        else:
            dte = 999  # Default high value if expiration not found
        
        # Calculate P&L percentage
        try:
            current_pnl = structure.calculate_pnl()
            max_profit = structure.max_profit if hasattr(structure, 'max_profit') else abs(structure.net_premium)
            
            if max_profit > 0:
                pnl_pct = current_pnl / max_profit
            else:
                pnl_pct = 0.0
        except:
            pnl_pct = 0.0
        
        # Exit conditions
        profit_target_hit = pnl_pct >= self.profit_target_pct
        stop_loss_hit = pnl_pct <= -self.stop_loss_pct
        dte_exit = dte <= self.exit_dte
        
        return profit_target_hit or stop_loss_hit or dte_exit
    
    def create_structure(self, market_data):
        """
        Create a short straddle structure from market data.
        
        Selects ATM strike with DTE in the target range.
        """
        option_chain = market_data.get('option_chain')
        if option_chain is None or option_chain.empty:
            return None
        
        current_date = market_data.get('date', datetime.now())
        spot = market_data.get('spot', 0.0)
        
        # Calculate DTE for each option
        option_chain = option_chain.copy()
        option_chain['dte'] = (pd.to_datetime(option_chain['expiration']) - current_date).dt.days
        
        # Filter by target DTE range
        target_dte_options = option_chain[
            (option_chain['dte'] >= self.min_entry_dte) & 
            (option_chain['dte'] <= self.max_entry_dte)
        ]
        
        if target_dte_options.empty:
            return None
        
        # Find ATM strike (closest to spot)
        target_dte_options['strike_diff'] = abs(target_dte_options['strike'] - spot)
        atm_strike = target_dte_options.loc[target_dte_options['strike_diff'].idxmin(), 'strike']
        
        # Get target expiration (closest to 37 DTE, middle of range)
        target_dte = 37
        target_dte_options['dte_diff'] = abs(target_dte_options['dte'] - target_dte)
        target_expiration = target_dte_options.loc[target_dte_options['dte_diff'].idxmin(), 'expiration']
        
        # Filter for ATM strike and target expiration
        atm_options = option_chain[
            (option_chain['strike'] == atm_strike) &
            (option_chain['expiration'] == target_expiration)
        ]
        
        # Get call and put
        calls = atm_options[atm_options['call_put'] == 'Call']
        puts = atm_options[atm_options['call_put'] == 'Put']
        
        if calls.empty or puts.empty:
            return None
        
        call_data = calls.iloc[0]
        put_data = puts.iloc[0]
        
        # Use mid price (average of bid and ask)
        call_price = (call_data['bid'] + call_data['ask']) / 2.0 if call_data['ask'] > 0 else call_data['bid']
        put_price = (put_data['bid'] + put_data['ask']) / 2.0 if put_data['ask'] > 0 else put_data['bid']
        
        # Create short straddle
        try:
            straddle = ShortStraddle.create(
                underlying=UNDERLYING,
                strike=atm_strike,
                expiration=pd.to_datetime(target_expiration).to_pydatetime(),
                call_price=call_price,
                put_price=put_price,
                quantity=self.contracts_per_trade,
                entry_date=current_date,
                underlying_price=spot,
                call_iv=call_data.get('vol', 0.20),
                put_iv=put_data.get('vol', 0.20)
            )
            return straddle
        except Exception as e:
            print(f'Failed to create straddle: {e}')
            return None

# Create strategy instance
strategy = AdvancedShortStraddleStrategy(
    name='AdvancedShortStraddle',
    initial_capital=INITIAL_CAPITAL
)

print(f'✓ Strategy created: {strategy.name}')
print(f'Parameters:')
print(f'  - IV Rank Threshold: {strategy.iv_rank_threshold}%')
print(f'  - Entry DTE Range: {strategy.min_entry_dte}-{strategy.max_entry_dte} days')
print(f'  - Exit DTE: {strategy.exit_dte} days')
print(f'  - Profit Target: {strategy.profit_target_pct:.0%}')
print(f'  - Stop Loss: {strategy.stop_loss_pct:.0%}')
print(f'  - Max Positions: {strategy._position_limits["max_positions"]}')

✓ Strategy created: AdvancedShortStraddle
Parameters:
  - IV Rank Threshold: 40.0%
  - Entry DTE Range: 3-70 days
  - Exit DTE: 3 days
  - Profit Target: 75%
  - Stop Loss: 200%
  - Max Positions: 10


## 4. Backtest Configuration

In [116]:
# Create data stream
print(f'Creating data stream for {UNDERLYING} from {START_DATE.date()} to {END_DATE.date()}')
data_stream = DataStream(
    data_source=adapter,
    start_date=START_DATE,
    end_date=END_DATE,
    underlying=UNDERLYING,
    dte_range=(3, 70),  # Look for options 3-70 days out
    cache_enabled=True,
    skip_missing_data=True
)

print(f'✓ Data stream created: {data_stream.num_trading_days} trading days')

# Create execution model
execution = ExecutionModel(
    commission_per_contract=0.65,
    slippage_pct=0.001  # 0.1% slippage
)

print(f'✓ Execution model created')
print(f'  - Commission: ${execution.commission_per_contract:.2f} per contract')

# Create backtest engine
engine = BacktestEngine(
    strategy=strategy,
    data_stream=data_stream,
    execution_model=execution,
    initial_capital=INITIAL_CAPITAL
)

print(f'✓ Backtest engine ready')
print(f'  - Initial Capital: ${INITIAL_CAPITAL:,.2f}')
print(f'  - Trading Days: {data_stream.num_trading_days}')

Creating data stream for SPY from 2023-01-04 to 2025-10-29
✓ Data stream created: 729 trading days
✓ Execution model created
  - Commission: $0.65 per contract
✓ Backtest engine ready
  - Initial Capital: $100,000.00
  - Trading Days: 729


## 5. Run Backtest

In [117]:
# Run the backtest
print('='*80)
print('RUNNING BACKTEST')
print('='*80)

results = engine.run()

print('='*80)
print('BACKTEST COMPLETE')
print('='*80)
print(f'Final Equity: ${results["final_equity"]:,.2f}')
print(f'Total Return: {results["total_return"]:.2%}')
print(f'Total Trades: {results["num_trades"]}')
print(f'Entries: {results["num_entries"]} | Exits: {results["num_exits"]}')

RUNNING BACKTEST


No option chain data found for SPY on 2023-01-05 with DTE range [3, 70]
No option chain data found for SPY on 2023-01-10 with DTE range [3, 70]
No option chain data found for SPY on 2023-01-12 with DTE range [3, 70]
No option chain data found for SPY on 2023-01-16 with DTE range [3, 70]
No option chain data found for SPY on 2023-01-17 with DTE range [3, 70]
No option chain data found for SPY on 2023-01-18 with DTE range [3, 70]
No option chain data found for SPY on 2023-01-19 with DTE range [3, 70]
Failed to execute entry: Adding position would exceed capital utilization limit (92.3% > 90.0%)
No option chain data found for SPY on 2023-01-24 with DTE range [3, 70]
Failed to execute entry: Adding position would exceed capital utilization limit (91.8% > 90.0%)
No option chain data found for SPY on 2023-01-26 with DTE range [3, 70]
No option chain data found for SPY on 2023-01-31 with DTE range [3, 70]
No option chain data found for SPY on 2023-02-02 with DTE range [3, 70]
No option chain 

BACKTEST COMPLETE
Final Equity: $160,145.28
Total Return: 60.15%
Total Trades: 223
Entries: 113 | Exits: 110


## 6. Calculate Comprehensive Metrics (30+)

In [118]:
# Calculate all metrics
metrics = engine.calculate_metrics(risk_free_rate=0.04)

# Display performance metrics
print('='*80)
print('PERFORMANCE METRICS')
print('='*80)
perf = metrics['performance']
# Note: total_return_pct is already a percentage (e.g., 25.0 for 25%), so use .2f not .2%
print(f"Total Return:           {perf.get('total_return_pct', 0):.2f}%")
# annualized_return is a decimal (e.g., 0.25 for 25%), so use .2%
print(f"Annualized Return:      {perf.get('annualized_return', 0):.2%}")
print(f"Volatility (Annual):    {perf.get('volatility', 0):.2%}")
print(f"Sharpe Ratio:           {perf.get('sharpe_ratio', 0):.3f}")
print(f"Sortino Ratio:          {perf.get('sortino_ratio', 0):.3f}")
print(f"Calmar Ratio:           {perf.get('calmar_ratio', 0):.3f}")
print(f"Max Drawdown:           {perf.get('max_drawdown', 0):.2%}")
print(f"Ulcer Index:            {perf.get('ulcer_index', 0):.4f}")

print('\n' + '='*80)
print('TRADE METRICS')
print('='*80)
print(f"Total Trades:           {perf.get('total_trades', 0)}")
# win_rate is already a percentage (0-100), so use .2f not .2%
print(f"Win Rate:               {perf.get('win_rate', 0):.2f}%")
print(f"Profit Factor:          {perf.get('profit_factor', 0):.3f}")
print(f"Expectancy:             ${perf.get('expectancy', 0):,.2f}")
print(f"Average Win:            ${perf.get('average_win', 0):,.2f}")
print(f"Average Loss:           ${perf.get('average_loss', 0):,.2f}")
print(f"Payoff Ratio:           {perf.get('payoff_ratio', 0):.3f}")
print(f"Max Consecutive Wins:   {perf.get('max_consecutive_wins', 0)}")
print(f"Max Consecutive Losses: {perf.get('max_consecutive_losses', 0)}")

print('\n' + '='*80)
print('RISK METRICS')
print('='*80)
risk = metrics['risk']
print(f"VaR (95%, Historical):  {risk.get('var_95_historical', 0):.4f}")
print(f"VaR (95%, Parametric):  {risk.get('var_95_parametric', 0):.4f}")
print(f"VaR (99%, Historical):  {risk.get('var_99_historical', 0):.4f}")
print(f"CVaR (95%):             {risk.get('cvar_95', 0):.4f}")
print(f"CVaR (99%):             {risk.get('cvar_99', 0):.4f}")
print(f"Skewness:               {risk.get('skewness', 0):.4f}")
print(f"Kurtosis:               {risk.get('kurtosis', 0):.4f}")
print(f"Downside Deviation:     {risk.get('downside_deviation', 0):.4f}")

print('\n' + '='*80)
print('DISTRIBUTION STATISTICS')
print('='*80)
dist = perf.get('returns_distribution', {})
print(f"Mean:                   {dist.get('mean', 0):.6f}")
print(f"Std Dev:                {dist.get('std', 0):.6f}")
print(f"Min:                    {dist.get('min', 0):.6f}")
print(f"25th Percentile:        {dist.get('25%', 0):.6f}")
print(f"Median:                 {dist.get('50%', 0):.6f}")
print(f"75th Percentile:        {dist.get('75%', 0):.6f}")
print(f"Max:                    {dist.get('max', 0):.6f}")

# Store for later use
equity_curve = results['equity_curve']
trade_log = results['trade_log']
greeks_history = results['greeks_history']

PERFORMANCE METRICS
Total Return:           60.15%
Annualized Return:      24.63%
Volatility (Annual):    17.61%
Sharpe Ratio:           1.111
Sortino Ratio:          1.723
Calmar Ratio:           2.714
Max Drawdown:           -9.07%
Ulcer Index:            0.0253

TRADE METRICS
Total Trades:           110
Win Rate:               78.18%
Profit Factor:          5.284
Expectancy:             $948.81
Average Win:            $1,496.86
Average Loss:           $-1,015.03
Payoff Ratio:           1.475
Max Consecutive Wins:   22
Max Consecutive Losses: 5

RISK METRICS
VaR (95%, Historical):  -0.0163
VaR (95%, Parametric):  -0.0173
VaR (99%, Historical):  -0.0317
CVaR (95%):             -0.0249
CVaR (99%):             -0.0385
Skewness:               0.4982
Kurtosis:               6.1449
Downside Deviation:     0.0072

DISTRIBUTION STATISTICS
Mean:                   0.000935
Std Dev:                0.011095
Min:                    -0.043610
25th Percentile:        0.000000
Median:               

## 7. Interactive Visualizations (8+)

Create publication-quality interactive charts using Plotly

In [119]:
# 1. Equity Curve with Drawdown
fig1 = make_subplots(
    rows=2, cols=1,
    subplot_titles=('Portfolio Equity', 'Drawdown'),
    vertical_spacing=0.1,
    row_heights=[0.7, 0.3]
)

# Equity curve
fig1.add_trace(
    go.Scatter(
        x=equity_curve.index,
        y=equity_curve['equity'],
        name='Equity',
        line=dict(color='#2E86AB', width=2)
    ),
    row=1, col=1
)

# Add initial capital line
fig1.add_hline(
    y=INITIAL_CAPITAL,
    line_dash="dash",
    line_color="gray",
    annotation_text="Initial Capital",
    row=1, col=1
)

# Calculate and plot drawdown
running_max = equity_curve['equity'].cummax()
drawdown = (equity_curve['equity'] - running_max) / running_max * 100

fig1.add_trace(
    go.Scatter(
        x=equity_curve.index,
        y=drawdown,
        name='Drawdown',
        fill='tozeroy',
        line=dict(color='#A23B72', width=1)
    ),
    row=2, col=1
)

fig1.update_xaxes(title_text="Date", row=2, col=1)
fig1.update_yaxes(title_text="Equity ($)", row=1, col=1)
fig1.update_yaxes(title_text="Drawdown (%)", row=2, col=1)

fig1.update_layout(
    title='Portfolio Performance',
    height=600,
    showlegend=True,
    template='plotly_white'
)

fig1.show()

print('Chart 1: Equity Curve with Drawdown ✓')

Chart 1: Equity Curve with Drawdown ✓


In [120]:
# 2. P&L Distribution
closed_trades = trade_log[trade_log['action'] == 'close']

if not closed_trades.empty:
    fig2 = go.Figure()
    
    # Histogram
    fig2.add_trace(go.Histogram(
        x=closed_trades['realized_pnl'],
        nbinsx=30,
        name='P&L Distribution',
        marker_color='#06A77D'
    ))
    
    # Add mean line
    mean_pnl = closed_trades['realized_pnl'].mean()
    fig2.add_vline(
        x=mean_pnl,
        line_dash="dash",
        line_color="red",
        annotation_text=f"Mean: ${mean_pnl:,.2f}"
    )
    
    fig2.update_layout(
        title='Trade P&L Distribution',
        xaxis_title='Realized P&L ($)',
        yaxis_title='Frequency',
        template='plotly_white',
        height=400
    )
    
    fig2.show()
    print('Chart 2: P&L Distribution ✓')
else:
    print('No closed trades for P&L distribution')

Chart 2: P&L Distribution ✓


In [121]:
# 3. Greeks Over Time
if not greeks_history.empty and len(greeks_history) > 0:
    fig3 = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Delta', 'Gamma', 'Theta', 'Vega'),
        vertical_spacing=0.12,
        horizontal_spacing=0.1
    )
    
    # Delta
    fig3.add_trace(
        go.Scatter(x=greeks_history.index, y=greeks_history['delta'], 
                   name='Delta', line=dict(color='#E63946')),
        row=1, col=1
    )
    
    # Gamma
    fig3.add_trace(
        go.Scatter(x=greeks_history.index, y=greeks_history['gamma'], 
                   name='Gamma', line=dict(color='#F77F00')),
        row=1, col=2
    )
    
    # Theta
    fig3.add_trace(
        go.Scatter(x=greeks_history.index, y=greeks_history['theta'], 
                   name='Theta', line=dict(color='#06A77D')),
        row=2, col=1
    )
    
    # Vega
    fig3.add_trace(
        go.Scatter(x=greeks_history.index, y=greeks_history['vega'], 
                   name='Vega', line=dict(color='#2E86AB')),
        row=2, col=2
    )
    
    fig3.update_layout(
        title='Portfolio Greeks Over Time',
        height=600,
        showlegend=False,
        template='plotly_white'
    )
    
    fig3.show()
    print('Chart 3: Greeks Over Time ✓')
else:
    print('No Greeks history available')

Chart 3: Greeks Over Time ✓


In [122]:
# 4. Trade Timeline
if not closed_trades.empty:
    fig4 = go.Figure()
    
    # Color code by profit/loss
    colors = ['green' if pnl > 0 else 'red' for pnl in closed_trades['realized_pnl']]
    
    fig4.add_trace(go.Scatter(
        x=closed_trades['timestamp'],
        y=closed_trades['realized_pnl'],
        mode='markers',
        marker=dict(
            size=10,
            color=colors,
            line=dict(width=1, color='white')
        ),
        text=[f"${pnl:,.2f}" for pnl in closed_trades['realized_pnl']],
        hovertemplate='<b>Date:</b> %{x}<br><b>P&L:</b> %{text}<extra></extra>'
    ))
    
    fig4.add_hline(y=0, line_dash="dash", line_color="gray")
    
    fig4.update_layout(
        title='Trade Timeline',
        xaxis_title='Date',
        yaxis_title='Realized P&L ($)',
        template='plotly_white',
        height=400
    )
    
    fig4.show()
    print('Chart 4: Trade Timeline ✓')
else:
    print('No closed trades for timeline')

Chart 4: Trade Timeline ✓


In [123]:
# 5. Rolling Sharpe Ratio
returns = equity_curve['equity'].pct_change().dropna()
window = 250  # 20-day rolling window

if len(returns) > window:
    rolling_sharpe = returns.rolling(window).mean() / returns.rolling(window).std() * np.sqrt(252)
    
    fig5 = go.Figure()
    
    fig5.add_trace(go.Scatter(
        x=rolling_sharpe.index,
        y=rolling_sharpe,
        name='Rolling Sharpe',
        line=dict(color='#2E86AB', width=2)
    ))
    
    fig5.add_hline(y=0, line_dash="dash", line_color="gray")
    fig5.add_hline(y=1, line_dash="dot", line_color="green", annotation_text="Sharpe = 1")
    
    fig5.update_layout(
        title=f'{window}-Day Rolling Sharpe Ratio',
        xaxis_title='Date',
        yaxis_title='Sharpe Ratio',
        template='plotly_white',
        height=400
    )
    
    fig5.show()
    print('Chart 5: Rolling Sharpe Ratio ✓')
else:
    print('Insufficient data for rolling Sharpe')

Chart 5: Rolling Sharpe Ratio ✓


In [124]:
# 6. Monthly Returns Heatmap
equity_df = equity_curve.copy()
equity_df['returns'] = equity_df['equity'].pct_change()
equity_df['year'] = equity_df.index.year
equity_df['month'] = equity_df.index.month

monthly_returns = equity_df.groupby(['year', 'month'])['returns'].sum().unstack()

if not monthly_returns.empty:
    fig6 = go.Figure(data=go.Heatmap(
        z=monthly_returns.values * 100,  # Convert to percentage
        x=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
        y=monthly_returns.index,
        colorscale='RdYlGn',
        text=monthly_returns.values * 100,
        texttemplate='%{text:.2f}%',
        textfont={"size": 10},
        colorbar=dict(title="Return (%)")
    ))
    
    fig6.update_layout(
        title='Monthly Returns Heatmap',
        xaxis_title='Month',
        yaxis_title='Year',
        template='plotly_white',
        height=400
    )
    
    fig6.show()
    print('Chart 6: Monthly Returns Heatmap ✓')
else:
    print('No monthly returns data')

Chart 6: Monthly Returns Heatmap ✓


In [125]:
# 7. Win/Loss Pie Chart
if not closed_trades.empty:
    wins = (closed_trades['realized_pnl'] > 0).sum()
    losses = (closed_trades['realized_pnl'] <= 0).sum()
    
    fig7 = go.Figure(data=[go.Pie(
        labels=['Wins', 'Losses'],
        values=[wins, losses],
        marker_colors=['#06A77D', '#E63946'],
        textinfo='label+percent',
        textfont_size=14,
        hole=0.3
    )])
    
    fig7.update_layout(
        title=f'Win Rate: {wins}/{wins+losses} ({wins/(wins+losses)*100:.1f}%)',
        template='plotly_white',
        height=400
    )
    
    fig7.show()
    print('Chart 7: Win/Loss Distribution ✓')
else:
    print('No trades for win/loss chart')

Chart 7: Win/Loss Distribution ✓


In [126]:
# 8. Cumulative Returns vs Buy & Hold
fig8 = go.Figure()

# Strategy cumulative returns
cum_returns_strategy = (equity_curve['equity'] / INITIAL_CAPITAL - 1) * 100

fig8.add_trace(go.Scatter(
    x=equity_curve.index,
    y=cum_returns_strategy,
    name='Short Straddle Strategy',
    line=dict(color='#2E86AB', width=2)
))

# Add zero line
fig8.add_hline(y=0, line_dash="dash", line_color="gray")

fig8.update_layout(
    title='Cumulative Returns',
    xaxis_title='Date',
    yaxis_title='Cumulative Return (%)',
    template='plotly_white',
    height=400
)

fig8.show()
print('Chart 8: Cumulative Returns ✓')

Chart 8: Cumulative Returns ✓


## 8. Strategy Insights & Analysis

In [127]:
print('='*80)
print('STRATEGY PERFORMANCE SUMMARY')
print('='*80)

summary = metrics['summary']

print(f"\nCapital:")
print(f"  Initial:     ${summary['initial_equity']:,.2f}")
print(f"  Final:       ${summary['final_equity']:,.2f}")
print(f"  Change:      ${summary['final_equity'] - summary['initial_equity']:,.2f}")

print(f"\nReturns:")
# total_return_pct is already a percentage (e.g., 25.0 for 25%), so use .2f not .2%
print(f"  Total:       {summary['total_return_pct']:.2f}%")
# annualized_return is a decimal (e.g., 0.25 for 25%), so use .2%
print(f"  Annualized:  {summary['annualized_return']:.2%}")

print(f"\nRisk-Adjusted:")
print(f"  Sharpe:      {summary['sharpe_ratio']:.3f}")
print(f"  Sortino:     {summary['sortino_ratio']:.3f}")
print(f"  Calmar:      {summary['calmar_ratio']:.3f}")

print(f"\nRisk:")
print(f"  Max DD:      {summary['max_drawdown']:.2%}")
print(f"  VaR (95%):   {summary['var_95']:.4f}")
print(f"  CVaR (95%):  {summary['cvar_95']:.4f}")
print(f"  Volatility:  {summary['volatility']:.2%}")

print(f"\nTrading:")
print(f"  Trades:      {summary['total_trades']}")
# win_rate is already a percentage (0-100), so use .2f not .2%
print(f"  Win Rate:    {summary['win_rate']:.2f}%")
print(f"  Profit Fac:  {summary['profit_factor']:.3f}")
print(f"  Expectancy:  ${summary['expectancy']:,.2f}")

print(f"\nPeriod:")
print(f"  Trading Days: {summary['trading_days']}")
print(f"  Start:        {START_DATE.date()}")
print(f"  End:          {END_DATE.date()}")

print('\n' + '='*80)
print('KEY INSIGHTS')
print('='*80)

# Generate insights based on metrics
insights = []

if summary['sharpe_ratio'] > 1.0:
    insights.append("✓ Strong risk-adjusted returns (Sharpe > 1.0)")
elif summary['sharpe_ratio'] > 0.5:
    insights.append("• Moderate risk-adjusted returns (Sharpe > 0.5)")
else:
    insights.append("✗ Weak risk-adjusted returns (Sharpe < 0.5)")

# win_rate is 0-100, so compare to 65, not 0.65
if summary['win_rate'] > 65:
    insights.append("✓ High win rate (>65%) - consistent performance")
elif summary['win_rate'] > 50:
    insights.append("• Moderate win rate (>50%)")
else:
    insights.append("✗ Low win rate (<50%) - strategy may need adjustment")

if summary['max_drawdown'] < -0.15:
    insights.append("✗ Significant drawdown (>15%) - high risk exposure")
elif summary['max_drawdown'] < -0.10:
    insights.append("• Moderate drawdown (10-15%)")
else:
    insights.append("✓ Low drawdown (<10%) - good risk management")

if summary['profit_factor'] > 1.5:
    insights.append("✓ Excellent profit factor (>1.5)")
elif summary['profit_factor'] > 1.0:
    insights.append("• Positive profit factor (>1.0)")
else:
    insights.append("✗ Poor profit factor (<1.0) - losses exceed wins")

for insight in insights:
    print(insight)

print('\n' + '='*80)
print('RECOMMENDATIONS')
print('='*80)

recommendations = [
    "1. Consider walk-forward analysis to validate strategy robustness",
    "2. Test sensitivity to IV rank threshold parameter",
    "3. Analyze exit timing - could profit targets be optimized?",
    "4. Evaluate correlation with VIX for regime filtering",
    "5. Consider scaling position size based on IV rank"
]

for rec in recommendations:
    print(rec)

STRATEGY PERFORMANCE SUMMARY

Capital:
  Initial:     $100,000.00
  Final:       $160,145.28
  Change:      $60,145.28

Returns:
  Total:       60.15%
  Annualized:  24.63%

Risk-Adjusted:
  Sharpe:      1.111
  Sortino:     1.723
  Calmar:      2.714

Risk:
  Max DD:      -9.07%
  VaR (95%):   -0.0163
  CVaR (95%):  -0.0249
  Volatility:  17.61%

Trading:
  Trades:      110
  Win Rate:    78.18%
  Profit Fac:  5.284
  Expectancy:  $948.81

Period:
  Trading Days: 540
  Start:        2023-01-04
  End:          2025-10-29

KEY INSIGHTS
✓ Strong risk-adjusted returns (Sharpe > 1.0)
✓ High win rate (>65%) - consistent performance
✓ Low drawdown (<10%) - good risk management
✓ Excellent profit factor (>1.5)

RECOMMENDATIONS
1. Consider walk-forward analysis to validate strategy robustness
2. Test sensitivity to IV rank threshold parameter
3. Analyze exit timing - could profit targets be optimized?
4. Evaluate correlation with VIX for regime filtering
5. Consider scaling position size based

## 9. Interactive Portfolio State Explorer

Explore the portfolio state throughout time with an interactive timeline showing:
- **Positions held** at each point in time
- **Cash balance** and equity
- **Open trades** with entry details and current P&L
- **Trade events** (entries and exits)

In [128]:
# =============================================================================
# Interactive Portfolio State Explorer
# =============================================================================
# This visualization allows you to explore the portfolio state at any point in time,
# showing open positions, cash balance, and trade history.

import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
import pandas as pd
import numpy as np

# Build portfolio state timeline from equity curve and trade log
def build_portfolio_timeline(equity_curve, trade_log):
    """
    Build a comprehensive timeline of portfolio states.
    
    Returns a DataFrame with columns:
    - timestamp: Date/time
    - equity: Total portfolio value
    - cash: Cash balance
    - positions_value: Value of open positions
    - num_positions: Number of open positions
    - open_positions: List of open position details
    """
    # Get unique timestamps from equity curve
    timeline = equity_curve.copy()
    
    # Track open positions over time
    open_positions_by_date = {}
    current_open = {}  # structure_id -> position info
    
    # Sort trade log by timestamp
    if not trade_log.empty:
        sorted_trades = trade_log.sort_values('timestamp')
        
        for _, trade in sorted_trades.iterrows():
            ts = trade['timestamp']
            structure_id = trade['structure_id']
            
            if trade['action'] == 'open':
                # Add to open positions
                current_open[structure_id] = {
                    'structure_id': structure_id,
                    'structure_type': trade.get('structure_type', 'unknown'),
                    'underlying': trade.get('underlying', 'SPY'),
                    'entry_date': ts,
                    'net_premium': trade.get('net_premium', 0),
                    'num_legs': trade.get('num_legs', 2),
                }
            elif trade['action'] == 'close':
                # Remove from open positions
                if structure_id in current_open:
                    del current_open[structure_id]
            
            # Store snapshot at this timestamp
            open_positions_by_date[ts] = list(current_open.values())
    
    # Build position count and details for each equity curve point
    position_counts = []
    position_details = []
    
    sorted_timestamps = sorted(open_positions_by_date.keys()) if open_positions_by_date else []
    
    for ts in timeline.index:
        # Find most recent position state at or before this timestamp
        latest_state = []
        for trade_ts in sorted_timestamps:
            if trade_ts <= ts:
                latest_state = open_positions_by_date[trade_ts]
            else:
                break
        
        position_counts.append(len(latest_state))
        position_details.append(latest_state)
    
    timeline['num_positions'] = position_counts
    timeline['open_positions'] = position_details
    
    return timeline

# Build the timeline
portfolio_timeline = build_portfolio_timeline(equity_curve, trade_log)

# Create interactive figure with multiple panels
fig = make_subplots(
    rows=4, cols=1,
    subplot_titles=(
        'Portfolio Equity & Cash',
        'Number of Open Positions',
        'Trade Events',
        'Position Value Breakdown'
    ),
    vertical_spacing=0.08,
    row_heights=[0.35, 0.2, 0.2, 0.25],
    specs=[[{"secondary_y": True}], [{}], [{}], [{}]]
)

# Panel 1: Equity and Cash over time
fig.add_trace(
    go.Scatter(
        x=portfolio_timeline.index,
        y=portfolio_timeline['equity'],
        name='Total Equity',
        line=dict(color='#2E86AB', width=2),
        hovertemplate='<b>Date:</b> %{x}<br><b>Equity:</b> $%{y:,.2f}<extra></extra>'
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=portfolio_timeline.index,
        y=portfolio_timeline['cash'],
        name='Cash',
        line=dict(color='#06A77D', width=1.5, dash='dot'),
        hovertemplate='<b>Date:</b> %{x}<br><b>Cash:</b> $%{y:,.2f}<extra></extra>'
    ),
    row=1, col=1
)

# Add positions value (equity - cash)
positions_value = portfolio_timeline['equity'] - portfolio_timeline['cash']
fig.add_trace(
    go.Scatter(
        x=portfolio_timeline.index,
        y=positions_value,
        name='Positions Value',
        line=dict(color='#F77F00', width=1.5),
        fill='tozeroy',
        fillcolor='rgba(247, 127, 0, 0.2)',
        hovertemplate='<b>Date:</b> %{x}<br><b>Positions Value:</b> $%{y:,.2f}<extra></extra>'
    ),
    row=1, col=1, secondary_y=True
)

# Panel 2: Number of open positions
fig.add_trace(
    go.Scatter(
        x=portfolio_timeline.index,
        y=portfolio_timeline['num_positions'],
        name='Open Positions',
        line=dict(color='#A23B72', width=2),
        fill='tozeroy',
        fillcolor='rgba(162, 59, 114, 0.3)',
        hovertemplate='<b>Date:</b> %{x}<br><b>Open Positions:</b> %{y}<extra></extra>'
    ),
    row=2, col=1
)

# Panel 3: Trade events (entries and exits)
if not trade_log.empty:
    entries = trade_log[trade_log['action'] == 'open']
    exits = trade_log[trade_log['action'] == 'close']
    
    # Entry markers
    if not entries.empty:
        fig.add_trace(
            go.Scatter(
                x=entries['timestamp'],
                y=[1] * len(entries),
                mode='markers',
                name='Entry',
                marker=dict(
                    symbol='triangle-up',
                    size=12,
                    color='#06A77D',
                    line=dict(width=1, color='white')
                ),
                text=[f"Entry: {row.get('structure_type', 'position')}<br>Premium: ${row.get('net_premium', 0):,.2f}" 
                      for _, row in entries.iterrows()],
                hovertemplate='%{text}<br><b>Date:</b> %{x}<extra></extra>'
            ),
            row=3, col=1
        )
    
    # Exit markers with P&L coloring
    if not exits.empty:
        exit_colors = ['#06A77D' if pnl > 0 else '#E63946' for pnl in exits['realized_pnl']]
        fig.add_trace(
            go.Scatter(
                x=exits['timestamp'],
                y=[0] * len(exits),
                mode='markers',
                name='Exit',
                marker=dict(
                    symbol='triangle-down',
                    size=12,
                    color=exit_colors,
                    line=dict(width=1, color='white')
                ),
                text=[f"Exit: ${row['realized_pnl']:,.2f} P&L<br>Reason: {row.get('exit_reason', 'unknown')}" 
                      for _, row in exits.iterrows()],
                hovertemplate='%{text}<br><b>Date:</b> %{x}<extra></extra>'
            ),
            row=3, col=1
        )

# Panel 4: Stacked area showing cash vs positions
fig.add_trace(
    go.Scatter(
        x=portfolio_timeline.index,
        y=portfolio_timeline['cash'],
        name='Cash Balance',
        fill='tozeroy',
        fillcolor='rgba(6, 167, 125, 0.5)',
        line=dict(color='#06A77D', width=1),
        stackgroup='one',
        hovertemplate='<b>Cash:</b> $%{y:,.2f}<extra></extra>'
    ),
    row=4, col=1
)

fig.add_trace(
    go.Scatter(
        x=portfolio_timeline.index,
        y=positions_value.clip(lower=0),  # Only positive values for stacking
        name='Positions (MTM)',
        fill='tonexty',
        fillcolor='rgba(247, 127, 0, 0.5)',
        line=dict(color='#F77F00', width=1),
        stackgroup='one',
        hovertemplate='<b>Positions Value:</b> $%{y:,.2f}<extra></extra>'
    ),
    row=4, col=1
)

# Update layout
fig.update_layout(
    title=dict(
        text='Interactive Portfolio State Explorer',
        font=dict(size=20)
    ),
    height=1000,
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    template='plotly_white',
    hovermode='x unified'
)

# Update axes labels
fig.update_yaxes(title_text="Value ($)", row=1, col=1)
fig.update_yaxes(title_text="Positions ($)", row=1, col=1, secondary_y=True)
fig.update_yaxes(title_text="# Positions", row=2, col=1)
fig.update_yaxes(title_text="Event", row=3, col=1, tickvals=[0, 1], ticktext=['Exit', 'Entry'])
fig.update_yaxes(title_text="Value ($)", row=4, col=1)
fig.update_xaxes(title_text="Date", row=4, col=1)

# Add range slider for easy navigation
fig.update_xaxes(
    rangeslider=dict(visible=True, thickness=0.05),
    row=4, col=1
)

fig.show()

print('Interactive Portfolio State Explorer ✓')
print(f'Timeline spans {len(portfolio_timeline)} trading days')
print(f'Total trades: {len(trade_log)} ({len(trade_log[trade_log["action"]=="open"])} entries, {len(trade_log[trade_log["action"]=="close"])} exits)')

Interactive Portfolio State Explorer ✓
Timeline spans 540 trading days
Total trades: 223 (113 entries, 110 exits)


In [129]:
# =============================================================================
# Detailed Trade Log & Position Table
# =============================================================================
# View all trades with detailed information about each position

# Create comprehensive trade summary table
def create_trade_summary(trade_log):
    """Create a detailed summary of all trades."""
    if trade_log.empty:
        return pd.DataFrame()
    
    entries = trade_log[trade_log['action'] == 'open'].copy()
    exits = trade_log[trade_log['action'] == 'close'].copy()
    
    # Match entries with exits by structure_id
    trade_summary = []
    
    for _, entry in entries.iterrows():
        struct_id = entry['structure_id']
        
        # Find matching exit
        matching_exit = exits[exits['structure_id'] == struct_id]
        
        trade_info = {
            'Trade #': len(trade_summary) + 1,
            'Entry Date': entry['timestamp'].strftime('%Y-%m-%d') if hasattr(entry['timestamp'], 'strftime') else str(entry['timestamp'])[:10],
            'Type': entry.get('structure_type', 'unknown').replace('_', ' ').title(),
            'Premium': f"${entry.get('net_premium', 0):,.2f}",
            'Exit Date': '-',
            'P&L': '-',
            'Return %': '-',
            'Hold Days': '-',
            'Exit Reason': '-',
            'Status': 'OPEN'
        }
        
        if not matching_exit.empty:
            exit_row = matching_exit.iloc[0]
            entry_date = entry['timestamp']
            exit_date = exit_row['timestamp']
            
            if hasattr(entry_date, 'date') and hasattr(exit_date, 'date'):
                hold_days = (exit_date - entry_date).days
            else:
                hold_days = 0
            
            pnl = exit_row.get('realized_pnl', 0)
            premium = entry.get('net_premium', 1)
            return_pct = (pnl / premium * 100) if premium > 0 else 0
            
            trade_info.update({
                'Exit Date': exit_date.strftime('%Y-%m-%d') if hasattr(exit_date, 'strftime') else str(exit_date)[:10],
                'P&L': f"${pnl:,.2f}",
                'Return %': f"{return_pct:+.1f}%",
                'Hold Days': hold_days,
                'Exit Reason': exit_row.get('exit_reason', 'unknown').replace('_', ' ').title(),
                'Status': 'WIN' if pnl > 0 else 'LOSS'
            })
        
        trade_summary.append(trade_info)
    
    return pd.DataFrame(trade_summary)

# Create the summary table
trade_summary_df = create_trade_summary(trade_log)

# Display using Plotly table for interactivity
if not trade_summary_df.empty:
    # Color code the status column
    status_colors = []
    for status in trade_summary_df['Status']:
        if status == 'WIN':
            status_colors.append('#06A77D')
        elif status == 'LOSS':
            status_colors.append('#E63946')
        else:
            status_colors.append('#F77F00')
    
    # Create table figure
    fig_table = go.Figure(data=[go.Table(
        header=dict(
            values=list(trade_summary_df.columns),
            fill_color='#2E86AB',
            font=dict(color='white', size=12),
            align='center',
            height=35
        ),
        cells=dict(
            values=[trade_summary_df[col] for col in trade_summary_df.columns],
            fill_color=[
                ['white'] * len(trade_summary_df),  # Trade #
                ['white'] * len(trade_summary_df),  # Entry Date
                ['white'] * len(trade_summary_df),  # Type
                ['white'] * len(trade_summary_df),  # Premium
                ['white'] * len(trade_summary_df),  # Exit Date
                [['rgba(6, 167, 125, 0.2)' if '$' in str(p) and '-' not in str(p) else 'rgba(230, 57, 70, 0.2)' if '-$' in str(p) else 'white' for p in trade_summary_df['P&L']]],  # P&L
                ['white'] * len(trade_summary_df),  # Return %
                ['white'] * len(trade_summary_df),  # Hold Days
                ['white'] * len(trade_summary_df),  # Exit Reason
                [[c.replace('#', 'rgba(').replace('06A77D', '6, 167, 125, 0.3)').replace('E63946', '230, 57, 70, 0.3)').replace('F77F00', '247, 127, 0, 0.3)') for c in status_colors]],  # Status
            ],
            font=dict(size=11),
            align='center',
            height=28
        )
    )])
    
    fig_table.update_layout(
        title=dict(
            text=f'Complete Trade Log ({len(trade_summary_df)} trades)',
            font=dict(size=18)
        ),
        height=min(600, 100 + len(trade_summary_df) * 30),
        margin=dict(l=20, r=20, t=60, b=20)
    )
    
    fig_table.show()
    
    # Print summary statistics
    print('\n' + '='*80)
    print('TRADE SUMMARY STATISTICS')
    print('='*80)
    
    closed_trades = trade_summary_df[trade_summary_df['Status'] != 'OPEN']
    if not closed_trades.empty:
        wins = len(closed_trades[closed_trades['Status'] == 'WIN'])
        losses = len(closed_trades[closed_trades['Status'] == 'LOSS'])
        
        # Calculate average hold time
        hold_days = closed_trades['Hold Days'].apply(lambda x: int(x) if str(x).isdigit() else 0)
        avg_hold = hold_days.mean()
        
        print(f"Closed Trades:     {len(closed_trades)}")
        print(f"Wins:              {wins} ({wins/len(closed_trades)*100:.1f}%)")
        print(f"Losses:            {losses} ({losses/len(closed_trades)*100:.1f}%)")
        print(f"Avg Hold Time:     {avg_hold:.1f} days")
    
    open_positions = trade_summary_df[trade_summary_df['Status'] == 'OPEN']
    if not open_positions.empty:
        print(f"\nOpen Positions:    {len(open_positions)}")
        for _, pos in open_positions.iterrows():
            print(f"  - {pos['Type']} opened {pos['Entry Date']}, Premium: {pos['Premium']}")

else:
    print('No trades to display')


TRADE SUMMARY STATISTICS
Closed Trades:     110
Wins:              86 (78.2%)
Losses:            24 (21.8%)
Avg Hold Time:     26.6 days

Open Positions:    3
  - Short Straddle opened 2025-10-13, Premium: $4,340.50
  - Short Straddle opened 2025-10-28, Premium: $3,243.00
  - Short Straddle opened 2025-10-29, Premium: $3,188.50


## Conclusion

This notebook demonstrates production-grade options backtesting with:
- ✓ Real historical market data from Dolt database
- ✓ Institutional-quality strategy implementation
- ✓ Comprehensive risk management
- ✓ 30+ performance and risk metrics
- ✓ 8+ interactive visualizations
- ✓ Full integration with backtesting framework

**Next Steps:**
1. Parameter optimization using grid search or Bayesian methods
2. Walk-forward analysis for out-of-sample validation
3. Multi-asset expansion (SPX, QQQ, IWM)
4. Portfolio-level hedging strategies
5. Live trading integration with broker API